In [1]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from keras import backend as K
import argparse
import os


# Ridle (with Universal Sentence Encoder)
In this notebook, the performance of "Ridle" is tested on different datasets. The neural network for further testing wasn't changed in any way, except giving it different inputs. Firstly the model is tested by itself to see how it performs in the normal case of just the representation from the RBM as inputs. Then we add our sentence embedding vectors to the representation vector and concatenate them to receive a new bigger vector with the size of input (50) and sentence embedding (384). Finally the whole program is also tested with only the sentence embeddings as inputs, to see how well the program can classify just by getting information from the summary embeddings.

In [ ]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [2]:
# GELU Activation function
def gelu(x):
    return 0.5 * x * (1 + K.tanh(x * 0.7978845608 * (1 + 0.044715 * x * x)))


In [65]:
df1["USE misclassified"] = ""
df1["ID list USE"] = ""

In [3]:
df1 = pd.read_pickle("./Misclassification.pkl")

### Loading and reading the data

In [349]:
parser = argparse.ArgumentParser(
    description='Instance Type Prediction using Ridle',
)
parser.add_argument('--dataset', nargs='?', default='Universities_DBpedia', type=str)
#parser.add_argument('--dataset', nargs='?', default='DBp_2016-04', type=str)
parser, unknown = parser.parse_known_args()

print('Loading mapping...')
if 'dbp' in parser.dataset.lower():
    mapping = pd.read_json('./dataset/dbp_type_mapping.json')
elif 'wd' in parser.dataset.lower() or 'wikidata' in parser.dataset.lower():
    mapping = pd.read_json('./dataset/wd_mapping_type.json')
else:
    mapping = pd.read_json('./dataset/{}/type_mapping.json'.format(parser.dataset))

# Load Representations
print('Reading Data...')
df = pd.read_csv('./dataset/{}/embedding.csv'.format(parser.dataset))
r = pd.merge(df, mapping, on='S')

Loading mapping...
Reading Data...


In [350]:
input_data = pd.read_pickle('./dataset/{}/input_data_unclean.pkl'.format(parser.dataset))

##If you couldn't create the unclean datasets:
#input_data = pd.read_pickle('./dataset/{}/input_data_summary_full.pkl'.format(parser.dataset))

In [351]:
non_empty = input_data[input_data.Summary!='']

In [352]:
r_copy = r.copy()

In [353]:
r_new = r_copy[r_copy.S.isin(non_empty['S'].values)]

In [354]:
input_new = input_data[input_data.S.isin(non_empty['S'].values)]

In [355]:
text = input_new['Summary']
sentences = ["" + temp + "" for temp in text]

In [356]:
inputs = r_new.drop(['S', 'Class'], axis=1).values

In [ ]:
r_new

#### Creating sentence embeddings and concatenated vectors with Universal Sentence Encoder
Right here we use USE....

In [357]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [358]:
logging.set_verbosity(logging.ERROR)

In [359]:
embeddings = embed(sentences)

In [360]:
embeddings = np.array(embeddings)
i = 0
dict_embeddings = {}
for entity in input_new["S"]:
    dict_embeddings[entity] = [embeddings[i]]
    i+=1

In [361]:
inputs = np.array(inputs)
ridle_dict = {}
i=0
for entity in r_new['S']:
    ridle_dict[entity] = [inputs[i]]
    i+=1

In [362]:
import numpy as np
ds = [ridle_dict, dict_embeddings]
d = {}
for k in ridle_dict.keys():
    d[k] = tuple(d[k] for d in ds)

In [363]:
conc_input =  []
for key in d:
    conc_input.append(np.concatenate((d[key][0][0], d[key][1][0])))
conc_input = np.array(conc_input)

In [ ]:
len(conc_input)

# Concatenated Vectors

In [364]:
K_FOLD = 10
mlb = MultiLabelBinarizer()
fold_no = 1
loss_per_fold, f1_macro, f1_micro, f1_weighted = [], [], [], []
kfold = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)
targets = mlb.fit_transform(r_new['Class'])
wrong_use = []
for train, test_new in kfold.split(conc_input, targets):
#     test_new = []
#     for x in range(len(test)):
#         if int(test[x]) in test_with_text:
#             test_new.append(test[x])
    model = Sequential()
    model.add(Dense(conc_input[train].shape[1], input_dim=conc_input[train].shape[1]))
    model.add(Activation(gelu, name='Gelu'))
    model.add(Dense(targets[train].shape[1], activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Training...')
    history = model.fit(conc_input[train], targets[train], batch_size=64, validation_data=(conc_input[test_new], targets[test_new]), epochs=100)
    y_pred = model.predict(conc_input[test_new])
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    
    # Generate F1 scores
    scores = model.evaluate(conc_input[test_new], targets[test_new], verbose=0)
    f1_macro.append(f1_score(targets[test_new], y_pred, average='macro', zero_division=1))
    f1_micro.append(f1_score(targets[test_new], y_pred, average='micro', zero_division=1))
    f1_weighted.append(f1_score(targets[test_new], y_pred, average='weighted', zero_division=1))
    
    for m in range(len(test_new)):
        comp = targets[test_new][m] ==  y_pred[m]
        if not(comp.all()):
            wrong_use.append((test_new[m], y_pred[m], targets[test_new][m]))
            #wrong.append(test_new[m])

    

    print('Score for fold', fold_no, ':', model.metrics_names[0], 'of', scores[0], ';', 'F1-Macro:', f1_macro[-1], 'F1-Micro:', f1_micro[-1])
    loss_per_fold.append(scores[0])

    fold_no += 1
    
# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_per_fold)):
    print('------------------------------------------------------------------------')
    print('> Fold', i+1, ' - Loss:', loss_per_fold[i], '- F1-Macro:', f1_macro[i], '%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('> F1-Macro:', np.mean(f1_macro), '(+-', np.std(f1_macro), ')')
print('> F1-Micro:', np.mean(f1_micro), '(+-', np.std(f1_micro), ')')
print('> Loss:', np.mean(loss_per_fold))
print('------------------------------------------------------------------------')

# Save results to file
result = {}
f1_macro = np.array(f1_macro)
f1_micro = np.array(f1_micro)
f1_weighted = np.array(f1_weighted)
result['F1-Macro'] = np.mean(f1_macro)
result['F1-Macro_std'] = np.std(f1_macro)
result['F1-Micro'] = np.mean(f1_micro)
result['F1-Micro_std'] = np.std(f1_micro)
result['F1-Weighted'] = np.mean(f1_weighted)
result['F1-Weighted_std'] = np.std(f1_weighted)
result['Dataset'] = parser.dataset
result['method'] = 'Concatenated Vectors'
df_result = pd.DataFrame([result])
print(df_result)

if os.path.isfile('./evaluation_instance_type.csv'):
    df_result.to_csv('./evaluation_instance_type.csv', mode='a', header=False, index=False)
# else:
    df_result.to_csv('./evaluation_instance_type.csv', index=False)

model.summary()

Training...
Epoch 1/100
94/94 [==============================] - 1s 5ms/step - loss: 0.0559 - accuracy: 0.3113 - val_loss: 0.0132 - val_accuracy: 0.6992
Epoch 2/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0100 - accuracy: 0.6302 - val_loss: 0.0130 - val_accuracy: 0.6060
Epoch 3/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0098 - accuracy: 0.5046 - val_loss: 0.0130 - val_accuracy: 0.5113
Epoch 4/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0094 - accuracy: 0.4048 - val_loss: 0.0129 - val_accuracy: 0.4481
Epoch 5/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0091 - accuracy: 0.5014 - val_loss: 0.0127 - val_accuracy: 0.4677
Epoch 6/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0087 - accuracy: 0.3731 - val_loss: 0.0129 - val_accuracy: 0.4902
Epoch 7/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0083 - accuracy: 0.3347 - val_loss: 0.0126 - val_accuracy: 0.3714
E

Epoch 58/100
94/94 [==============================] - 0s 4ms/step - loss: 4.2080e-05 - accuracy: 0.1532 - val_loss: 0.0390 - val_accuracy: 0.2030
Epoch 59/100
94/94 [==============================] - 0s 4ms/step - loss: 3.9043e-05 - accuracy: 0.1554 - val_loss: 0.0401 - val_accuracy: 0.1835
Epoch 60/100
94/94 [==============================] - 0s 4ms/step - loss: 3.8123e-05 - accuracy: 0.1551 - val_loss: 0.0397 - val_accuracy: 0.2045
Epoch 61/100
94/94 [==============================] - 0s 4ms/step - loss: 3.3740e-05 - accuracy: 0.1626 - val_loss: 0.0409 - val_accuracy: 0.1850
Epoch 62/100
94/94 [==============================] - 0s 4ms/step - loss: 3.3564e-05 - accuracy: 0.1442 - val_loss: 0.0405 - val_accuracy: 0.2045
Epoch 63/100
94/94 [==============================] - 0s 4ms/step - loss: 3.2542e-05 - accuracy: 0.1409 - val_loss: 0.0405 - val_accuracy: 0.1774
Epoch 64/100
94/94 [==============================] - 0s 4ms/step - loss: 3.0407e-05 - accuracy: 0.1534 - val_loss: 0.0417 -

Epoch 14/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0050 - accuracy: 0.0211 - val_loss: 0.0070 - val_accuracy: 0.0316
Epoch 15/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0047 - accuracy: 0.0249 - val_loss: 0.0080 - val_accuracy: 0.0812
Epoch 16/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0043 - accuracy: 0.0199 - val_loss: 0.0070 - val_accuracy: 0.0030
Epoch 17/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0040 - accuracy: 0.0353 - val_loss: 0.0070 - val_accuracy: 0.0105
Epoch 18/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0033 - accuracy: 0.0227 - val_loss: 0.0074 - val_accuracy: 0.0015
Epoch 19/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0033 - accuracy: 0.0378 - val_loss: 0.0091 - val_accuracy: 0.0361
Epoch 20/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0031 - accuracy: 0.0361 - val_loss: 0.0083 - val_accuracy: 0.0135
Epoch 

94/94 [==============================] - 0s 3ms/step - loss: 0.0012 - accuracy: 0.4735 - val_loss: 0.0413 - val_accuracy: 0.5549
Epoch 27/100
94/94 [==============================] - 0s 3ms/step - loss: 9.8498e-04 - accuracy: 0.5370 - val_loss: 0.0415 - val_accuracy: 0.4150
Epoch 28/100
94/94 [==============================] - 0s 3ms/step - loss: 9.5320e-04 - accuracy: 0.4137 - val_loss: 0.0401 - val_accuracy: 0.4812
Epoch 29/100
94/94 [==============================] - 0s 3ms/step - loss: 7.6361e-04 - accuracy: 0.5008 - val_loss: 0.0428 - val_accuracy: 0.5278
Epoch 30/100
94/94 [==============================] - 0s 3ms/step - loss: 6.7567e-04 - accuracy: 0.5805 - val_loss: 0.0446 - val_accuracy: 0.4812
Epoch 31/100
94/94 [==============================] - 0s 3ms/step - loss: 5.7915e-04 - accuracy: 0.4837 - val_loss: 0.0443 - val_accuracy: 0.4947
Epoch 32/100
94/94 [==============================] - 0s 3ms/step - loss: 6.4885e-04 - accuracy: 0.4882 - val_loss: 0.0457 - val_accuracy: 0.

94/94 [==============================] - 0s 3ms/step - loss: 4.2079e-04 - accuracy: 0.3848 - val_loss: 0.0084 - val_accuracy: 0.3895
Epoch 39/100
94/94 [==============================] - 0s 3ms/step - loss: 3.2920e-04 - accuracy: 0.3776 - val_loss: 0.0084 - val_accuracy: 0.4165
Epoch 40/100
94/94 [==============================] - 0s 3ms/step - loss: 2.6024e-04 - accuracy: 0.4142 - val_loss: 0.0086 - val_accuracy: 0.3684
Epoch 41/100
94/94 [==============================] - 0s 4ms/step - loss: 2.1883e-04 - accuracy: 0.3794 - val_loss: 0.0086 - val_accuracy: 0.3519
Epoch 42/100
94/94 [==============================] - 0s 3ms/step - loss: 1.9196e-04 - accuracy: 0.4177 - val_loss: 0.0087 - val_accuracy: 0.3504
Epoch 43/100
94/94 [==============================] - 0s 3ms/step - loss: 1.6991e-04 - accuracy: 0.3614 - val_loss: 0.0089 - val_accuracy: 0.3880
Epoch 44/100
94/94 [==============================] - 0s 3ms/step - loss: 1.5001e-04 - accuracy: 0.4020 - val_loss: 0.0091 - val_accuracy

...

Epoch 18/100
94/94 [==============================] - 0s 3ms/step - loss: 0.0037 - accuracy: 0.0120 - val_loss: 0.0112 - val_accuracy: 0.0226
Epoch 19/100
94/94 [==============================] - 0s 3ms/step - loss: 0.0033 - accuracy: 0.0197 - val_loss: 0.0116 - val_accuracy: 0.0030
Epoch 20/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0031 - accuracy: 0.0095 - val_loss: 0.0109 - val_accuracy: 0.0090
Epoch 21/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0026 - accuracy: 0.0070 - val_loss: 0.0112 - val_accuracy: 0.0105
Epoch 22/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0024 - accuracy: 0.0154 - val_loss: 0.0119 - val_accuracy: 0.0075
Epoch 23/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0021 - accuracy: 0.0084 - val_loss: 0.0122 - val_accuracy: 0.0150
Epoch 24/100
94/94 [==============================] - 0s 4ms/step - loss: 0.0021 - accuracy: 0.0159 - val_loss: 0.0122 - val_accuracy: 0.0030
Epoch 

94/94 [==============================] - 0s 3ms/step - loss: 9.4201e-04 - accuracy: 0.1611 - val_loss: 0.0220 - val_accuracy: 0.1278
Epoch 31/100
94/94 [==============================] - 0s 4ms/step - loss: 8.3999e-04 - accuracy: 0.1454 - val_loss: 0.0227 - val_accuracy: 0.1654
Epoch 32/100
94/94 [==============================] - 0s 3ms/step - loss: 8.5806e-04 - accuracy: 0.1622 - val_loss: 0.0219 - val_accuracy: 0.2662
Epoch 33/100
94/94 [==============================] - 0s 3ms/step - loss: 6.0686e-04 - accuracy: 0.2057 - val_loss: 0.0233 - val_accuracy: 0.1805
Epoch 34/100
94/94 [==============================] - 0s 3ms/step - loss: 5.4715e-04 - accuracy: 0.1442 - val_loss: 0.0236 - val_accuracy: 0.1504
Epoch 35/100
94/94 [==============================] - 0s 3ms/step - loss: 5.1894e-04 - accuracy: 0.1749 - val_loss: 0.0238 - val_accuracy: 0.2075
Epoch 36/100
94/94 [==============================] - 0s 3ms/step - loss: 4.0383e-04 - accuracy: 0.1561 - val_loss: 0.0255 - val_accuracy

94/94 [==============================] - 0s 3ms/step - loss: 1.7814e-04 - accuracy: 0.2152 - val_loss: 0.0127 - val_accuracy: 0.2451
Epoch 43/100
94/94 [==============================] - 0s 3ms/step - loss: 1.4881e-04 - accuracy: 0.2718 - val_loss: 0.0128 - val_accuracy: 0.2902
Epoch 44/100
94/94 [==============================] - 0s 3ms/step - loss: 1.7022e-04 - accuracy: 0.2994 - val_loss: 0.0124 - val_accuracy: 0.2376
Epoch 45/100
94/94 [==============================] - 0s 3ms/step - loss: 1.7165e-04 - accuracy: 0.2454 - val_loss: 0.0134 - val_accuracy: 0.2090
Epoch 46/100
94/94 [==============================] - 0s 3ms/step - loss: 1.1343e-04 - accuracy: 0.2523 - val_loss: 0.0134 - val_accuracy: 0.2406
Epoch 47/100
94/94 [==============================] - 0s 3ms/step - loss: 1.1680e-04 - accuracy: 0.2434 - val_loss: 0.0132 - val_accuracy: 0.2135
Epoch 48/100
94/94 [==============================] - 0s 3ms/step - loss: 1.2112e-04 - accuracy: 0.2239 - val_loss: 0.0137 - val_accuracy

94/94 [==============================] - 0s 3ms/step - loss: 4.9341e-05 - accuracy: 0.5400 - val_loss: 0.0302 - val_accuracy: 0.4917
Epoch 55/100
94/94 [==============================] - 0s 3ms/step - loss: 4.7818e-05 - accuracy: 0.5489 - val_loss: 0.0300 - val_accuracy: 0.5083
Epoch 56/100
94/94 [==============================] - 0s 3ms/step - loss: 3.9198e-05 - accuracy: 0.5363 - val_loss: 0.0301 - val_accuracy: 0.5248
Epoch 57/100
94/94 [==============================] - 0s 3ms/step - loss: 3.5916e-05 - accuracy: 0.5245 - val_loss: 0.0311 - val_accuracy: 0.5023
Epoch 58/100
94/94 [==============================] - 0s 3ms/step - loss: 3.4856e-05 - accuracy: 0.5497 - val_loss: 0.0303 - val_accuracy: 0.5053
Epoch 59/100
94/94 [==============================] - 0s 3ms/step - loss: 3.2658e-05 - accuracy: 0.5210 - val_loss: 0.0305 - val_accuracy: 0.5023
Epoch 60/100
94/94 [==============================] - 0s 3ms/step - loss: 3.0434e-05 - accuracy: 0.5163 - val_loss: 0.0313 - val_accuracy

Model: "sequential_140"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_280 (Dense)           (None, 562)               316406    
                                                                 
 Gelu (Activation)           (None, 562)               0         
                                                                 
 dense_281 (Dense)           (None, 12)                6756      
                                                                 
Total params: 323,162
Trainable params: 323,162
Non-trainable params: 0
_________________________________________________________________


In [365]:
wrong_id_bert = []
for element in wrong_sbert:
    wrong_id_bert.append(element[0])
    wrong_id_bert.sort()

NameError: name 'wrong_use' is not defined

In [341]:
pred_bert = []
for element in wrong_sbert:
    pred_bert.append(element[1])

In [342]:
targets_bert = []
for element in wrong_sbert:
    targets_bert.append(element[2])

In [343]:
df1.at['{}'.format(parser.dataset), "USE misclassified"] = len(wrong_sbert)
df1.at['{}'.format(parser.dataset), "ID list USE"] = wrong_id_bert

In [344]:
df1

,0,Ridle misclassified,Sbert misclassified,Misclassified in both,ID list Bert,Prediction Bert,Target Bert,ID list Ridle,Prediction Ridle,Target Ridle,USE misclassified,ID list USE
Books_DBpedia,Books_DBpedia,22,23,"{4366, 6548, 5665, 6438, 1959, 1065, 7340, 811...","[311, 1065, 1401, 1959, 1985, 2047, 3337, 3441...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1065, 1401, 1959, 1985, 2047, 2925, 3441, 436...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",21,"[1065, 1401, 1959, 1985, 2047, 3441, 4366, 496..."
ChemicalCompounds_DBpedia,ChemicalCompounds_DBpedia,6,8,"{2406, 5286, 7216, 1906, 4186, 3452}","[1906, 2406, 3452, 4186, 5286, 5323, 7216, 7929]","[[0.0, 1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0....","[[0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0...","[1906, 2406, 3452, 4186, 5286, 7216]","[[0.0, 1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0....","[[0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1...",8,"[1906, 2406, 3452, 4186, 5286, 5323, 7216, 7929]"
Company_DBpedia,Company_DBpedia,142,133,"{3587, 7172, 4101, 7180, 3089, 6677, 4632, 259...","[133, 156, 182, 218, 489, 716, 818, 1068, 1114...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,...","[62, 119, 123, 133, 156, 182, 291, 489, 607, 6...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,...",155,"[124, 132, 133, 489, 660, 662, 716, 743, 818, ..."
DBp_2016-04,DBp_2016-04,5959,4179,"{8193, 16389, 16391, 8201, 32788, 32789, 24602...","[646, 1282, 1414, 1603, 1703, 1736, 2247, 2269...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[646, 1282, 1414, 1599, 1603, 1655, 1682, 1703...",,,4173,"[646, 1282, 1603, 1661, 1668, 1703, 2269, 2463..."
Person_DBpedia,Person_DBpedia,2831,2606,"{8192, 2, 24580, 4, 24585, 12, 19, 24606, 32, ...","[216, 261, 274, 278, 474, 852, 880, 963, 1087,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[216, 261, 274, 278, 775, 852, 880, 963, 1246,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2505,"[216, 261, 274, 474, 963, 1087, 1246, 1457, 14..."
Songs_DBpedia,Songs_DBpedia,151,181,"{0, 2560, 3073, 3588, 6, 1545, 3082, 3601, 361...","[0, 6, 9, 35, 80, 121, 142, 149, 168, 196, 199...","[[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]...","[[0, 0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 0, 1, ...","[0, 6, 80, 163, 168, 199, 223, 248, 260, 293, ...","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]...","[[0, 0, 0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 0, 1, ...",203,"[0, 6, 9, 27, 59, 63, 80, 142, 163, 168, 196, ..."
umls,umls,52,45,"{0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...","[0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",38,"[0, 1, 2, 3, 4, 5, 6, 9, 11, 12, 13, 14, 16, 1..."
Universities_DBpedia,Universities_DBpedia,35,37,"{1272, 4362, 652, 1165, 4625, 4755, 3991, 3993...","[304, 462, 548, 652, 1095, 1120, 1133, 1165, 1...","[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,...","[[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...","[462, 548, 652, 843, 1015, 1095, 1120, 1133, 1...","[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,...","[[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...",37,"[462, 548, 652, 1095, 1120, 1133, 1162, 1165, ..."
Books_Dbpedia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [345]:
df1.to_pickle("./Misclassification.pkl")

In [346]:
false_negative = {}
false_positive = {}
for i in range(len(pred_bert)):
    for j in range(len(pred_bert[i])):
        if targets_bert[i][j] == 1 and pred_bert[i][j] == 0:
            if mlb.classes_[j] in false_negative.keys():
                false_negative[mlb.classes_[j]].append(r['S'].iloc[wrong_id_bert[i]])
            else: 
                false_negative[mlb.classes_[j]] = [r['S'].iloc[wrong_id_bert[i]]] 
            
        elif targets_bert[i][j] == 0 and pred_bert[i][j] == 1:
            if mlb.classes_[j] in false_positive.keys():
                false_positive[mlb.classes_[j]].append(r['S'].iloc[wrong_id_bert[i]])
            else: 
                false_positive[mlb.classes_[j]] = [r['S'].iloc[wrong_id_bert[i]]]

In [347]:
fn = pd.DataFrame.from_dict(false_negative, orient='index')
fn.to_pickle("./false_negative_USE/'{}'.pkl".format(parser.dataset))

In [348]:
fp = pd.DataFrame.from_dict(false_positive, orient='index')
fp.to_pickle("./false_positive_USE/'{}'.pkl".format(parser.dataset))

# Only Sentence Embeddings

In [20]:
K_FOLD = 10
mlb = MultiLabelBinarizer()
fold_no = 1
loss_per_fold, f1_macro, f1_micro, f1_weighted = [], [], [], []
kfold = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)
targets = mlb.fit_transform(input_new['Class'])
#emb_array = r.drop(['S', 'Class'], axis=1).values
for train, test_new in kfold.split(emb_array, targets):
#     test_new = []
#     for x in range(len(test)):
#         if int(test[x]) in test_with_text:
#             test_new.append(test[x])
    model = Sequential()
    model.add(Dense(emb_array[train].shape[1], input_dim=emb_array[train].shape[1]))
    model.add(Activation(gelu, name='Gelu'))
    model.add(Dense(targets[train].shape[1], activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Training...')
    history = model.fit(emb_array[train], targets[train], batch_size=64, validation_data=(emb_array[test_new], targets[test_new]), epochs=100)
    y_pred = model.predict(emb_array[test_new])
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    
    # Generate F1 scores
    scores = model.evaluate(emb_array[test_new], targets[test_new], verbose=0)
    f1_macro.append(f1_score(targets[test_new], y_pred, average='macro', zero_division=1))
    f1_micro.append(f1_score(targets[test_new], y_pred, average='micro', zero_division=1))
    f1_weighted.append(f1_score(targets[test_new], y_pred, average='weighted', zero_division=1))

    print('Score for fold', fold_no, ':', model.metrics_names[0], 'of', scores[0], ';', 'F1-Macro:', f1_macro[-1], 'F1-Micro:', f1_micro[-1])
    loss_per_fold.append(scores[0])

    fold_no += 1
    
# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_per_fold)):
    print('------------------------------------------------------------------------')
    print('> Fold', i+1, ' - Loss:', loss_per_fold[i], '- F1-Macro:', f1_macro[i], '%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('> F1-Macro:', np.mean(f1_macro), '(+-', np.std(f1_macro), ')')
print('> F1-Micro:', np.mean(f1_micro), '(+-', np.std(f1_micro), ')')
print('> Loss:', np.mean(loss_per_fold))
print('------------------------------------------------------------------------')

# Save results to file
result = {}
f1_macro = np.array(f1_macro)
f1_micro = np.array(f1_micro)
f1_weighted = np.array(f1_weighted)
result['F1-Macro'] = np.mean(f1_macro)
result['F1-Macro_std'] = np.std(f1_macro)
result['F1-Micro'] = np.mean(f1_micro)
result['F1-Micro_std'] = np.std(f1_micro)
result['F1-Weighted'] = np.mean(f1_weighted)
result['F1-Weighted_std'] = np.std(f1_weighted)
result['Dataset'] = parser.dataset
result['method'] = 'Only Embeddings'
df_result = pd.DataFrame([result])
print(df_result)

if os.path.isfile('./evaluation_instance_type.csv'):
    df_result.to_csv('./evaluation_instance_type.csv', mode='a', header=False, index=False)
# else:
    df_result.to_csv('./evaluation_instance_type.csv', index=False)

    
model.summary()

Training...
Epoch 1/100
421/421 [==============================] - 2s 3ms/step - loss: 0.0626 - accuracy: 0.9774 - val_loss: 0.0230 - val_accuracy: 0.9870
Epoch 2/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0190 - accuracy: 0.9881 - val_loss: 0.0166 - val_accuracy: 0.9970
Epoch 3/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0146 - accuracy: 0.9793 - val_loss: 0.0139 - val_accuracy: 0.9823
Epoch 4/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0125 - accuracy: 0.9192 - val_loss: 0.0125 - val_accuracy: 0.9729
Epoch 5/100
421/421 [==============================] - 1s 2ms/step - loss: 0.0113 - accuracy: 0.7604 - val_loss: 0.0116 - val_accuracy: 0.9890
Epoch 6/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0105 - accuracy: 0.7746 - val_loss: 0.0111 - val_accuracy: 0.9104
Epoch 7/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0099 - accuracy: 0.7378 - val_loss: 0.0107 - val_accu

Epoch 58/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0012 - accuracy: 0.9084 - val_loss: 0.0150 - val_accuracy: 0.8876
Epoch 59/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0012 - accuracy: 0.9031 - val_loss: 0.0151 - val_accuracy: 0.8900
Epoch 60/100
421/421 [==============================] - 2s 4ms/step - loss: 0.0011 - accuracy: 0.8996 - val_loss: 0.0153 - val_accuracy: 0.8930
Epoch 61/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0010 - accuracy: 0.8963 - val_loss: 0.0156 - val_accuracy: 0.9070
Epoch 62/100
421/421 [==============================] - 1s 3ms/step - loss: 9.3318e-04 - accuracy: 0.8990 - val_loss: 0.0158 - val_accuracy: 0.8742
Epoch 63/100
421/421 [==============================] - 1s 3ms/step - loss: 8.7388e-04 - accuracy: 0.8870 - val_loss: 0.0160 - val_accuracy: 0.8883
Epoch 64/100
421/421 [==============================] - 1s 3ms/step - loss: 8.1208e-04 - accuracy: 0.8846 - val_loss: 0.0163 - v

421/421 [==============================] - 1s 3ms/step - loss: 0.0080 - accuracy: 0.6893 - val_loss: 0.0096 - val_accuracy: 0.8552
Epoch 14/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0078 - accuracy: 0.7106 - val_loss: 0.0095 - val_accuracy: 0.5371
Epoch 15/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0076 - accuracy: 0.7040 - val_loss: 0.0094 - val_accuracy: 0.9441
Epoch 16/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0074 - accuracy: 0.8118 - val_loss: 0.0093 - val_accuracy: 0.9254
Epoch 17/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0072 - accuracy: 0.8206 - val_loss: 0.0094 - val_accuracy: 0.7224
Epoch 18/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0071 - accuracy: 0.7695 - val_loss: 0.0096 - val_accuracy: 0.7043
Epoch 19/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0069 - accuracy: 0.8244 - val_loss: 0.0094 - val_accuracy: 0.8114
Epoch

Epoch 70/100
421/421 [==============================] - 1s 3ms/step - loss: 5.2240e-04 - accuracy: 0.7906 - val_loss: 0.0177 - val_accuracy: 0.8164
Epoch 71/100
421/421 [==============================] - 1s 3ms/step - loss: 4.8358e-04 - accuracy: 0.7895 - val_loss: 0.0184 - val_accuracy: 0.8264
Epoch 72/100
421/421 [==============================] - 1s 3ms/step - loss: 4.4568e-04 - accuracy: 0.7937 - val_loss: 0.0180 - val_accuracy: 0.7512
Epoch 73/100
421/421 [==============================] - 1s 3ms/step - loss: 4.1346e-04 - accuracy: 0.7734 - val_loss: 0.0184 - val_accuracy: 0.7803
Epoch 74/100
421/421 [==============================] - 1s 3ms/step - loss: 3.7920e-04 - accuracy: 0.7780 - val_loss: 0.0187 - val_accuracy: 0.7495
Epoch 75/100
421/421 [==============================] - 1s 3ms/step - loss: 3.5021e-04 - accuracy: 0.7664 - val_loss: 0.0190 - val_accuracy: 0.7288
Epoch 76/100
421/421 [==============================] - 1s 3ms/step - loss: 3.2102e-04 - accuracy: 0.7597 - val_

421/421 [==============================] - 1s 3ms/step - loss: 2.3433e-04 - accuracy: 0.7148 - val_loss: 0.0195 - val_accuracy: 0.6933
Epoch 82/100
421/421 [==============================] - 1s 3ms/step - loss: 1.9600e-04 - accuracy: 0.7172 - val_loss: 0.0197 - val_accuracy: 0.7211
Epoch 83/100
421/421 [==============================] - 1s 3ms/step - loss: 1.8515e-04 - accuracy: 0.7096 - val_loss: 0.0198 - val_accuracy: 0.6759
Epoch 84/100
421/421 [==============================] - 1s 3ms/step - loss: 1.7936e-04 - accuracy: 0.6977 - val_loss: 0.0201 - val_accuracy: 0.6829
Epoch 85/100
421/421 [==============================] - 1s 3ms/step - loss: 1.6203e-04 - accuracy: 0.6890 - val_loss: 0.0206 - val_accuracy: 0.7007
Epoch 86/100
421/421 [==============================] - 1s 3ms/step - loss: 1.5666e-04 - accuracy: 0.6929 - val_loss: 0.0207 - val_accuracy: 0.6625
Epoch 87/100
421/421 [==============================] - 1s 3ms/step - loss: 1.4613e-04 - accuracy: 0.6680 - val_loss: 0.0210 

...

Epoch 15/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0077 - accuracy: 0.7863 - val_loss: 0.0091 - val_accuracy: 0.7197
Epoch 16/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0075 - accuracy: 0.7645 - val_loss: 0.0090 - val_accuracy: 0.6274
Epoch 17/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0073 - accuracy: 0.7880 - val_loss: 0.0090 - val_accuracy: 0.6247
Epoch 18/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0071 - accuracy: 0.7705 - val_loss: 0.0089 - val_accuracy: 0.6478
Epoch 19/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0070 - accuracy: 0.8378 - val_loss: 0.0089 - val_accuracy: 0.7124
Epoch 20/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0068 - accuracy: 0.8347 - val_loss: 0.0090 - val_accuracy: 0.7475
Epoch 21/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0066 - accuracy: 0.8159 - val_loss: 0.0089 - val_accuracy:

421/421 [==============================] - 1s 2ms/step - loss: 0.0058 - accuracy: 0.8866 - val_loss: 0.0089 - val_accuracy: 0.8946
Epoch 27/100
421/421 [==============================] - 1s 2ms/step - loss: 0.0056 - accuracy: 0.8580 - val_loss: 0.0091 - val_accuracy: 0.8699
Epoch 28/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0054 - accuracy: 0.8827 - val_loss: 0.0093 - val_accuracy: 0.8444
Epoch 29/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0053 - accuracy: 0.8785 - val_loss: 0.0091 - val_accuracy: 0.8421
Epoch 30/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0051 - accuracy: 0.8560 - val_loss: 0.0091 - val_accuracy: 0.8628
Epoch 31/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0049 - accuracy: 0.8690 - val_loss: 0.0091 - val_accuracy: 0.8528
Epoch 32/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0048 - accuracy: 0.8840 - val_loss: 0.0091 - val_accuracy: 0.8632
Epoch

421/421 [==============================] - 1s 3ms/step - loss: 0.0039 - accuracy: 0.8748 - val_loss: 0.0094 - val_accuracy: 0.9100
Epoch 38/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0038 - accuracy: 0.8856 - val_loss: 0.0095 - val_accuracy: 0.8421
Epoch 39/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0036 - accuracy: 0.8948 - val_loss: 0.0096 - val_accuracy: 0.8772
Epoch 40/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0035 - accuracy: 0.8932 - val_loss: 0.0096 - val_accuracy: 0.8909
Epoch 41/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0033 - accuracy: 0.8984 - val_loss: 0.0098 - val_accuracy: 0.9307
Epoch 42/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.9145 - val_loss: 0.0098 - val_accuracy: 0.8662
Epoch 43/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0030 - accuracy: 0.9019 - val_loss: 0.0099 - val_accuracy: 0.9117
Epoch

Epoch 93/100
421/421 [==============================] - 1s 3ms/step - loss: 1.0234e-04 - accuracy: 0.7879 - val_loss: 0.0210 - val_accuracy: 0.7688
Epoch 94/100
421/421 [==============================] - 1s 3ms/step - loss: 1.0156e-04 - accuracy: 0.7844 - val_loss: 0.0211 - val_accuracy: 0.7554
Epoch 95/100
421/421 [==============================] - 1s 3ms/step - loss: 8.7102e-05 - accuracy: 0.7773 - val_loss: 0.0214 - val_accuracy: 0.7534
Epoch 96/100
421/421 [==============================] - 1s 4ms/step - loss: 7.5606e-05 - accuracy: 0.7829 - val_loss: 0.0217 - val_accuracy: 0.7651
Epoch 97/100
421/421 [==============================] - 1s 4ms/step - loss: 7.2038e-05 - accuracy: 0.7695 - val_loss: 0.0218 - val_accuracy: 0.7387
Epoch 98/100
421/421 [==============================] - 2s 4ms/step - loss: 8.1995e-05 - accuracy: 0.7631 - val_loss: 0.0224 - val_accuracy: 0.7715
Epoch 99/100
421/421 [==============================] - 1s 3ms/step - loss: 7.6912e-05 - accuracy: 0.7669 - val_

Epoch 49/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0022 - accuracy: 0.9101 - val_loss: 0.0124 - val_accuracy: 0.9164
Epoch 50/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0021 - accuracy: 0.9136 - val_loss: 0.0128 - val_accuracy: 0.9067
Epoch 51/100
421/421 [==============================] - 1s 2ms/step - loss: 0.0020 - accuracy: 0.9153 - val_loss: 0.0127 - val_accuracy: 0.8856
Epoch 52/100
421/421 [==============================] - 1s 2ms/step - loss: 0.0019 - accuracy: 0.9048 - val_loss: 0.0133 - val_accuracy: 0.8809
Epoch 53/100
421/421 [==============================] - 1s 2ms/step - loss: 0.0018 - accuracy: 0.9080 - val_loss: 0.0133 - val_accuracy: 0.9013
Epoch 54/100
421/421 [==============================] - 1s 2ms/step - loss: 0.0017 - accuracy: 0.9067 - val_loss: 0.0134 - val_accuracy: 0.8812
Epoch 55/100
421/421 [==============================] - 1s 2ms/step - loss: 0.0016 - accuracy: 0.8999 - val_loss: 0.0140 - val_accuracy: